In [2]:
import math

import ipywidgets as widgets
import jupyter_cadquery as jcq
from jupyter_cadquery.replay import replay, show

from build123d import *

Overwriting auto display for cadquery Workplane and Shape


In [3]:
jcq.set_defaults(
    control="orbit",
    axes=True,
    axes0=True,
    grid=(True, False, False),
    glass=True,
    theme="browser",
)

# open_viewer("Cadquery")
replay_box = widgets.Checkbox(False, description="Enable Replay")
out = widgets.Output()
display(replay_box)
display(out)


@out.capture()
def changed(e):
    if e["new"]:
        jcq.replay.enable_replay(warning=False)
    else:
        jcq.replay.disable_replay()


replay_box.observe(changed, names="value")

Checkbox(value=False, description='Enable Replay')

Output()

In [4]:
layer_height = 0.2
handle_height = 15
handle_length = 40
reinforcement_length = 0
thickness = 1.5

In [5]:
with BuildLine() as side:
    bottom = Line((0, 0), (handle_length, 0))
    top = Line(bottom @ 0 + (10, handle_height), bottom @ 1 + (0, handle_height))
    c = Line(bottom @ 0, top @ 0)
    Offset(bottom, c, top, amount=thickness / 2, kind=Kind.INTERSECTION)

show(side.line)

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…

In [6]:
with BuildSketch() as side_sk:
    with Locations((thickness, thickness / 2)):
        Add(side.line)
        MakeFace()
    Rectangle(
        handle_length + thickness,
        handle_height + thickness,
        align=(Align.MIN, Align.MIN),
        mode=Mode.INTERSECT,
    )

show(side_sk.sketch)

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…

In [9]:
with BuildPart() as out:
    Add(side_sk.sketch)
    Extrude(amount=handle_length, both=True)
    Box(
        handle_length,
        handle_height,
        thickness,
        align=(Align.MIN, Align.MIN, Align.CENTER),
    )
    with Workplanes(Plane.XZ):
        with BuildSketch() as mask:
            RegularPolygon(handle_length, side_count=4)
            RegularPolygon(
                handle_length - 10 / math.sin(math.radians(45)),
                side_count=4,
                mode=Mode.SUBTRACT,
            )
        Extrude(amount=-(handle_height + thickness), mode=Mode.INTERSECT)

show(out.part)
out.part.export_step("/tmp/x.step")

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : /tmp/x.step(1096 ents)  Write  Done


<IFSelect_ReturnStatus.IFSelect_RetDone: 1>

In [41]:
with BuildPart() as reinforcement:
    with BuildSketch() as a:
        width = (handle_length + reinforcement_length)*2+10
        with Locations(Rotation(0,0,45)):
            Rectangle(width, width)
            Rectangle(width - 10*2, width - 10*2, mode=Mode.SUBTRACT)
    Extrude(amount=layer_height*3)
    with Locations(Rotation(10, 0, 0)):
        with BuildSketch():
            Rectangle(width*2, width*2)
        Extrude(amount=width, mode=Mode.SUBTRACT)

show(reinforcement.part)
reinforcement.part.export_step("/tmp/x2.step")

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : /tmp/x2.step(490 ents)  Write  Done


<IFSelect_ReturnStatus.IFSelect_RetDone: 1>